In [1]:
!pip install lifelines
!pip install plotly==4.14.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 349 kB 4.8 MB/s 
     |████████████████████████████████| 77 kB 2.5 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=13acf20e54a3026a67cdcb3d691049a0462a2a4ea2e6efa9102504f2c1c9fd4b
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.5 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test, multivariate_logrank_test

pd.options.plotting.backend = 'plotly'

In [4]:
df=pd.read_csv('churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,MonthLevel,TotalLevel,TenureLevel,ChurnBinary
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,Month-to-month,Yes,Electronic check,29.85,29.85,No,Low,Very Low,New,0.0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,One year,No,Mailed check,56.95,1889.50,No,Low,Moderate,Loyal,0.0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Low,Very Low,New,1.0
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,One year,No,Bank transfer (automatic),42.30,1840.75,No,Low,Moderate,Loyal,0.0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,Moderate,Very Low,New,1.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


In [30]:
grupo1=df[df['Partner']=='Yes']
grupo2=df[df['Partner']=='No']

grupo2.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,MonthLevel,TotalLevel,TenureLevel,ChurnBinary
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,One year,No,Mailed check,56.95,1889.50,No,Low,Moderate,Loyal,0.0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Low,Very Low,New,1.0
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,One year,No,Bank transfer (automatic),42.30,1840.75,No,Low,Moderate,Loyal,0.0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,Moderate,Very Low,New,1.0
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,Very High,Low,Regular,1.0


In [31]:
T=grupo1['tenure']
E=grupo1['ChurnBinary']

model= KaplanMeierFitter() # y=mx+b
model.fit(T,E, label='Female')

<lifelines.KaplanMeierFitter:"Female", fitted with 3393 total observations, 2724 right-censored observations>

In [32]:
fig1=model.survival_function_
fig1.plot()

In [33]:
T2= grupo2['tenure']
E2= grupo2['ChurnBinary']

model2=KaplanMeierFitter()
model2.fit(T2,E2, label='Male')

fig2=model2.survival_function_



In [13]:
type(fig2)

pandas.core.frame.DataFrame

In [34]:
grafica=pd.concat([fig1,fig2],axis=1)
grafica

,Female,Male
timeline,,
0.0,1.000000,1.000000
1.0,0.980548,0.913713
2.0,0.974596,0.883587
3.0,0.967693,0.862205
4.0,0.961040,0.843293
...,...,...
68.0,0.731741,0.498761
69.0,0.727005,0.483175
70.0,0.716392,0.469102


In [35]:
grafica.plot()

In [36]:
#logrank test
test=logrank_test(T,T2, event_observed_A=E ,event_observed_B=E2)
test.print_summary()


<lifelines.StatisticalResult: logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 1
         test_name = logrank_test

---
 test_statistic      p  -log2(p)
         423.54 <0.005    310.21

In [54]:
#hacer una función que acepte un DF y lo filtre automáticamente por un parámetro
#que nosotros demos.
def survival(data:pd.DataFrame,
              group_field:str,time_field:str,event_field:str ):
  model=KaplanMeierFitter()
  results=[]

  for i in data[group_field].unique():
    group=data[data[group_field]==i]
    T=group[time_field]
    E=group[event_field]
    model.fit(T,E,label=str(i))
    results.append(model.survival_function_)
  return pd.concat(results,axis=1)



In [55]:
df_surv=survival(df,'Contract','tenure','ChurnBinary')

In [56]:
df_surv.plot()